In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame

# Specify the file path
file_path = "../Data/prosperLoanData.csv"

In [3]:
# Read the CSV file into a Pandas DataFrame
Uncleaned_df = pd.read_csv(file_path)

# Review the DataFrame

Uncleaned_df.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20


In [4]:
#Creating Credit Score ( Lowest + Highest credit score /2 )
Uncleaned_df['Average Credit Score'] = (Uncleaned_df['CreditScoreRangeLower'] + Uncleaned_df['CreditScoreRangeUpper']) / 2

In [5]:
#types of columns 
specific_columns = ['LoanStatus','Term','ProsperRating (numeric)','Average Credit Score','EmploymentStatus','IsBorrowerHomeowner','StatedMonthlyIncome',
'BorrowerRate','EmploymentStatusDuration','LoanMonthsSinceOrigination','MonthlyLoanPayment','RevolvingCreditBalance','LoanOriginalAmount','LP_CustomerPayments'  ] 

In [6]:
specific_columns_uncleaned_df = Uncleaned_df[specific_columns]
specific_columns_uncleaned_df.head(10)

,LoanStatus,Term,ProsperRating (numeric),Average Credit Score,EmploymentStatus,IsBorrowerHomeowner,StatedMonthlyIncome,BorrowerRate,EmploymentStatusDuration,LoanMonthsSinceOrigination,MonthlyLoanPayment,RevolvingCreditBalance,LoanOriginalAmount,LP_CustomerPayments
0,Completed,36,NaN,649.5,Self-employed,True,3083.333333,0.1580,2.0,78,330.43,0.0,9425,11396.14
1,Current,36,6.0,689.5,Employed,False,6125.000000,0.0920,44.0,0,318.93,3989.0,10000,0.00
2,Completed,36,NaN,489.5,Not available,False,2083.333333,0.2750,NaN,86,123.32,NaN,3001,4186.63
3,Current,36,6.0,809.5,Employed,True,2875.000000,0.0974,113.0,16,321.45,1444.0,10000,5143.20
4,Current,36,3.0,689.5,Employed,True,9583.333333,0.2085,44.0,6,563.97,6193.0,15000,2819.85
5,Current,60,5.0,749.5,Employed,True,8333.333333,0.1314,82.0,3,342.37,62999.0,15000,679.34
6,Current,36,2.0,689.5,Employed,False,2083.333333,0.2712,172.0,11,122.67,5812.0,3000,1226.70
7,Current,36,4.0,709.5,Employed,False,3355.750000,0.2019,103.0,10,372.60,1260.0,10000,3353.40
8,Current,36,7.0,829.5,Employed,True,3333.333333,0.0629,269.0,3,305.54,9906.0,10000,611.08
9,Current,36,7.0,829.5,Employed,True,3333.333333,0.0629,269.0,3,305.54,9906.0,10000,611.08


In [7]:
specific_columns_uncleaned_df = specific_columns_uncleaned_df.dropna()

In [8]:
#changing data that is not a value to be NA
specific_columns_uncleaned_df.replace("Not displayed", pd.NA, inplace=True)


In [9]:
#changing data in Employment Status to be numbers
EmploymentStatus_unique_values = specific_columns_uncleaned_df['EmploymentStatus'].unique()
mapping = {
    'Self-employed': 1,
    'Employed': 2,
    'Not available': 3,
    'Full-time': 4,
    'Other': 5,
    'Not employed': 6,
    'Part-time': 7,
    'Retired': 8
}
# Replace the categorical values with numerical values
specific_columns_uncleaned_df['EmploymentStatus'] = specific_columns_uncleaned_df['EmploymentStatus'].replace(mapping)
specific_columns_uncleaned_df

C:\Users\User\AppData\Local\Temp\ipykernel_11116\3225507980.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  specific_columns_uncleaned_df['EmploymentStatus'] = specific_columns_uncleaned_df['EmploymentStatus'].replace(mapping)


,LoanStatus,Term,ProsperRating (numeric),Average Credit Score,EmploymentStatus,IsBorrowerHomeowner,StatedMonthlyIncome,BorrowerRate,EmploymentStatusDuration,LoanMonthsSinceOrigination,MonthlyLoanPayment,RevolvingCreditBalance,LoanOriginalAmount,LP_CustomerPayments
1,Current,36,6.0,689.5,2,False,6125.000000,0.0920,44.0,0,318.93,3989.0,10000,0.0000
3,Current,36,6.0,809.5,2,True,2875.000000,0.0974,113.0,16,321.45,1444.0,10000,5143.2000
4,Current,36,3.0,689.5,2,True,9583.333333,0.2085,44.0,6,563.97,6193.0,15000,2819.8500
5,Current,60,5.0,749.5,2,True,8333.333333,0.1314,82.0,3,342.37,62999.0,15000,679.3400
6,Current,36,2.0,689.5,2,False,2083.333333,0.2712,172.0,11,122.67,5812.0,3000,1226.7000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113932,Current,36,4.0,709.5,2,True,4333.333333,0.1864,246.0,11,364.74,7714.0,10000,3647.4000
113933,FinalPaymentInProgress,36,6.0,709.5,2,True,8041.666667,0.1110,21.0,28,65.57,15743.0,2000,2330.5500
113934,Current,60,3.0,709.5,2,True,2875.000000,0.2150,84.0,3,273.35,22147.0,10000,546.7000
113935,Completed,60,4.0,689.5,4,True,3875.000000,0.2605,94.0,28,449.55,11956.0,15000,21122.5600


In [10]:
LoanStatus_unique_values = specific_columns_uncleaned_df['LoanStatus'].unique()
print(LoanStatus_unique_values)

['Current' 'Past Due (1-15 days)' 'Defaulted' 'Completed' 'Chargedoff'
 'Past Due (16-30 days)' 'Past Due (61-90 days)' 'Past Due (31-60 days)'
 'Past Due (91-120 days)' 'FinalPaymentInProgress' 'Past Due (>120 days)']


In [11]:
mapping = {
    'Current': 0,
    'Completed': 0,
    'FinalPaymentInProgress': 0,
    'Past Due (1-15 days)': 1,
    'Past Due (16-30 days)': 1,
    'Past Due (31-60 days)': 1,
    'Past Due (61-90 days)': 2,
    'Past Due (91-120 days)': 2,
    'Past Due (>120 days)': 2,
    'Chargedoff':2,
    'Defaulted':2,
}
# Replace the categorical values with numerical values
specific_columns_uncleaned_df['LoanStatus'] = specific_columns_uncleaned_df['LoanStatus'].replace(mapping)
specific_columns_uncleaned_df

C:\Users\User\AppData\Local\Temp\ipykernel_11116\4087781651.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  specific_columns_uncleaned_df['LoanStatus'] = specific_columns_uncleaned_df['LoanStatus'].replace(mapping)


,LoanStatus,Term,ProsperRating (numeric),Average Credit Score,EmploymentStatus,IsBorrowerHomeowner,StatedMonthlyIncome,BorrowerRate,EmploymentStatusDuration,LoanMonthsSinceOrigination,MonthlyLoanPayment,RevolvingCreditBalance,LoanOriginalAmount,LP_CustomerPayments
1,0,36,6.0,689.5,2,False,6125.000000,0.0920,44.0,0,318.93,3989.0,10000,0.0000
3,0,36,6.0,809.5,2,True,2875.000000,0.0974,113.0,16,321.45,1444.0,10000,5143.2000
4,0,36,3.0,689.5,2,True,9583.333333,0.2085,44.0,6,563.97,6193.0,15000,2819.8500
5,0,60,5.0,749.5,2,True,8333.333333,0.1314,82.0,3,342.37,62999.0,15000,679.3400
6,0,36,2.0,689.5,2,False,2083.333333,0.2712,172.0,11,122.67,5812.0,3000,1226.7000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113932,0,36,4.0,709.5,2,True,4333.333333,0.1864,246.0,11,364.74,7714.0,10000,3647.4000
113933,0,36,6.0,709.5,2,True,8041.666667,0.1110,21.0,28,65.57,15743.0,2000,2330.5500
113934,0,60,3.0,709.5,2,True,2875.000000,0.2150,84.0,3,273.35,22147.0,10000,546.7000
113935,0,60,4.0,689.5,4,True,3875.000000,0.2605,94.0,28,449.55,11956.0,15000,21122.5600


In [12]:
specific_columns_uncleaned_df = pd.DataFrame(specific_columns_uncleaned_df, columns=['LoanStatus','Term','ProsperRating (numeric)','Average Credit Score','EmploymentStatus','IsBorrowerHomeowner','StatedMonthlyIncome',
'BorrowerRate','EmploymentStatusDuration','LoanMonthsSinceOrigination','MonthlyLoanPayment','RevolvingCreditBalance','LoanOriginalAmount','LP_CustomerPayments'])

In [13]:
specific_columns_uncleaned_df['IsBorrowerHomeowner'] = specific_columns_uncleaned_df['IsBorrowerHomeowner'].map({True: 1, False: 0})

In [14]:
specific_columns_uncleaned_df.head()

,LoanStatus,Term,ProsperRating (numeric),Average Credit Score,EmploymentStatus,IsBorrowerHomeowner,StatedMonthlyIncome,BorrowerRate,EmploymentStatusDuration,LoanMonthsSinceOrigination,MonthlyLoanPayment,RevolvingCreditBalance,LoanOriginalAmount,LP_CustomerPayments
1,0,36,6.0,689.5,2,0,6125.000000,0.0920,44.0,0,318.93,3989.0,10000,0.00
3,0,36,6.0,809.5,2,1,2875.000000,0.0974,113.0,16,321.45,1444.0,10000,5143.20
4,0,36,3.0,689.5,2,1,9583.333333,0.2085,44.0,6,563.97,6193.0,15000,2819.85
5,0,60,5.0,749.5,2,1,8333.333333,0.1314,82.0,3,342.37,62999.0,15000,679.34
6,0,36,2.0,689.5,2,0,2083.333333,0.2712,172.0,11,122.67,5812.0,3000,1226.70


In [15]:
specific_columns_uncleaned_df.dtypes

LoanStatus                      int64
Term                            int64
ProsperRating (numeric)       float64
Average Credit Score          float64
EmploymentStatus                int64
IsBorrowerHomeowner             int64
StatedMonthlyIncome           float64
BorrowerRate                  float64
EmploymentStatusDuration      float64
LoanMonthsSinceOrigination      int64
MonthlyLoanPayment            float64
RevolvingCreditBalance        float64
LoanOriginalAmount              int64
LP_CustomerPayments           float64
dtype: object

In [16]:
columns_to_scale = ['LoanStatus' ,
                    'Term',
                     'ProsperRating (numeric)',
                     'Average Credit Score',
                     'EmploymentStatus',
                     'IsBorrowerHomeowner',
                     'StatedMonthlyIncome',
                     'BorrowerRate',
                     'EmploymentStatusDuration',
                     'LoanMonthsSinceOrigination',
                     'MonthlyLoanPayment',
                     'RevolvingCreditBalance',
                     'LoanOriginalAmount',
                     'LP_CustomerPayments']

In [17]:
specific_columns_uncleaned_df = specific_columns_uncleaned_df.dropna(subset=columns_to_scale)

In [18]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(specific_columns_uncleaned_df[columns_to_scale])

In [19]:
scaled_df = pd.DataFrame(scaled_data,columns=[columns_to_scale])

In [20]:
scaled_df

,LoanStatus,Term,ProsperRating (numeric),Average Credit Score,EmploymentStatus,IsBorrowerHomeowner,StatedMonthlyIncome,BorrowerRate,EmploymentStatusDuration,LoanMonthsSinceOrigination,MonthlyLoanPayment,RevolvingCreditBalance,LoanOriginalAmount,LP_CustomerPayments
0,-0.324123,-0.557261,1.152074,-0.411842,-0.328162,-1.058752,0.023500,-1.393753,-0.608327,-1.152967,0.144557,-0.444937,0.145693,-0.866623
1,-0.324123,-0.557261,1.152074,2.136159,-0.328162,0.944508,-0.370888,-1.321398,0.102440,-0.001558,0.158056,-0.526144,0.145693,0.348122
2,-0.324123,-0.557261,-0.640879,-0.411842,-0.328162,0.944508,0.443168,0.167245,-0.608327,-0.721189,1.457209,-0.374610,0.940903,-0.200618
3,-0.324123,1.504434,0.554423,0.862158,-0.328162,0.944508,0.291481,-0.865828,-0.216890,-0.937078,0.270123,1.437989,0.940903,-0.706174
4,-0.324123,-0.557261,-1.238529,-0.411842,-0.328162,-1.058752,-0.466957,1.007371,0.710197,-0.361374,-0.906786,-0.386768,-0.967602,-0.576896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84829,-0.324123,-0.557261,-0.043228,0.012825,-0.328162,0.944508,-0.193919,-0.128875,1.472468,-0.361374,0.389956,-0.326077,0.145693,-0.005163
84830,-0.324123,-0.557261,1.152074,0.012825,-0.328162,0.944508,0.256087,-1.139170,-0.845249,0.861998,-1.212664,-0.069883,-1.126644,-0.316183
84831,-0.324123,1.504434,-0.640879,0.012825,-0.328162,0.944508,-0.370888,0.254340,-0.196288,-0.937078,-0.099610,0.134459,0.145693,-0.737501
84832,-0.324123,1.504434,-0.043228,-0.411842,1.606733,0.944508,-0.249538,0.864000,-0.093279,0.861998,0.844274,-0.190721,0.940903,4.122202


In [21]:
# Create a list to store inertia values
inertia = []

# Create a list to store the values of k
k = list(range(1, 11))

In [22]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the spread_df DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(scaled_df)
    inertia.append(k_model.inertia_)

In [23]:
# Create a Dictionary that holds the list values for k and inertia
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame using the elbow_data Dictionary
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,1,1.187676e+06
1,2,9.900002e+05
2,3,9.489202e+05
3,4,8.809534e+05
4,5,8.173287e+05


In [24]:
import hvplot.pandas


df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)

:Curve   [k]   (inertia)

In [25]:
# Define the model with the lower value of k clusters
# Use a random_state of 1 to generate the model
model = KMeans(n_clusters=2, random_state=1)

# Fit the model
model.fit(scaled_df)

# Make predictions
k = model.predict(scaled_df)

# Create a copy of the DataFrame and name it as spread_df_predictions
spread_df_predictions = scaled_df.copy()

# Add a class column with the labels to the spread_df_predictions DataFrame
spread_df_predictions['clusters'] = k

In [26]:
spread_df_predictions

,LoanStatus,Term,ProsperRating (numeric),Average Credit Score,EmploymentStatus,IsBorrowerHomeowner,StatedMonthlyIncome,BorrowerRate,EmploymentStatusDuration,LoanMonthsSinceOrigination,MonthlyLoanPayment,RevolvingCreditBalance,LoanOriginalAmount,LP_CustomerPayments,clusters
0,-0.324123,-0.557261,1.152074,-0.411842,-0.328162,-1.058752,0.023500,-1.393753,-0.608327,-1.152967,0.144557,-0.444937,0.145693,-0.866623,1
1,-0.324123,-0.557261,1.152074,2.136159,-0.328162,0.944508,-0.370888,-1.321398,0.102440,-0.001558,0.158056,-0.526144,0.145693,0.348122,1
2,-0.324123,-0.557261,-0.640879,-0.411842,-0.328162,0.944508,0.443168,0.167245,-0.608327,-0.721189,1.457209,-0.374610,0.940903,-0.200618,1
3,-0.324123,1.504434,0.554423,0.862158,-0.328162,0.944508,0.291481,-0.865828,-0.216890,-0.937078,0.270123,1.437989,0.940903,-0.706174,1
4,-0.324123,-0.557261,-1.238529,-0.411842,-0.328162,-1.058752,-0.466957,1.007371,0.710197,-0.361374,-0.906786,-0.386768,-0.967602,-0.576896,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84829,-0.324123,-0.557261,-0.043228,0.012825,-0.328162,0.944508,-0.193919,-0.128875,1.472468,-0.361374,0.389956,-0.326077,0.145693,-0.005163,1
84830,-0.324123,-0.557261,1.152074,0.012825,-0.328162,0.944508,0.256087,-1.139170,-0.845249,0.861998,-1.212664,-0.069883,-1.126644,-0.316183,1
84831,-0.324123,1.504434,-0.640879,0.012825,-0.328162,0.944508,-0.370888,0.254340,-0.196288,-0.937078,-0.099610,0.134459,0.145693,-0.737501,1
84832,-0.324123,1.504434,-0.043228,-0.411842,1.606733,0.944508,-0.249538,0.864000,-0.093279,0.861998,0.844274,-0.190721,0.940903,4.122202,1


In [27]:
print(spread_df_predictions.columns)

MultiIndex([(                'LoanStatus',),
            (                      'Term',),
            (   'ProsperRating (numeric)',),
            (      'Average Credit Score',),
            (          'EmploymentStatus',),
            (       'IsBorrowerHomeowner',),
            (       'StatedMonthlyIncome',),
            (              'BorrowerRate',),
            (  'EmploymentStatusDuration',),
            ('LoanMonthsSinceOrigination',),
            (        'MonthlyLoanPayment',),
            (    'RevolvingCreditBalance',),
            (        'LoanOriginalAmount',),
            (       'LP_CustomerPayments',),
            (                  'clusters',)],
           )


In [28]:
# Reset the index completely to convert to a single index DataFrame
spread_df_predictions_single_index = spread_df_predictions.reset_index(drop=True)

# Create a new DataFrame with a single index and columns as strings
spread_df_predictions_strings = pd.DataFrame(data=spread_df_predictions_single_index.values, columns=spread_df_predictions_single_index.columns.map(str))

# Now the new DataFrame has a single index with columns as strings
print(spread_df_predictions_strings)

       ('LoanStatus',)  ('Term',)  ('ProsperRating (numeric)',)  \
0            -0.324123  -0.557261                      1.152074   
1            -0.324123  -0.557261                      1.152074   
2            -0.324123  -0.557261                     -0.640879   
3            -0.324123   1.504434                      0.554423   
4            -0.324123  -0.557261                     -1.238529   
...                ...        ...                           ...   
84829        -0.324123  -0.557261                     -0.043228   
84830        -0.324123  -0.557261                      1.152074   
84831        -0.324123   1.504434                     -0.640879   
84832        -0.324123   1.504434                     -0.043228   
84833        -0.324123  -0.557261                      1.152074   

       ('Average Credit Score',)  ('EmploymentStatus',)  \
0                      -0.411842              -0.328162   
1                       2.136159              -0.328162   
2                 

In [29]:
num_indices = spread_df_predictions_strings.index.nlevels

print("Number of indices in the DataFrame:", num_indices)

Number of indices in the DataFrame: 1


In [ ]:
import xgboost as xgb

data = spread_df_predictions_strings
label = 